----
# <center>Dataset creation

In [1]:
import os
from pathlib import Path
import matlab.engine
import numpy as np
import shutil
import tempfile
import json

base_dir = Path("./")

 # --- Start MATLAB Engine ---
eng = matlab.engine.start_matlab()

 # --- Add SCOPE path to MATLAB's search path ---
scope_path = './SCOPE'  # e.g., '/home/user/Documents/SCOPE'
eng.addpath(scope_path, nargout=0)

# MODTRAN paths
modtran_dir = base_dir / "MODTRAN5"
modtran_exe = modtran_dir / "bin" / "Mod5_mac.exe" 
modtran_tp5_template = modtran_dir / "HyPlant-FLUO_Modtran5_base_v1.tp5"

# SCOPE paths
scope_dir = base_dir / "SCOPE"
scope_main = scope_dir / "SCOPE.m"
scope_wrapper = scope_dir / "run_scope_wrapper.m"

# Output directory
output_dir = base_dir / "synthetic_dataset"
output_dir.mkdir(exist_ok=True)

print(f"Checking path: {scope_dir.resolve()}")
print(f"Directory exists: {scope_dir.exists()}")
print(f"Directory contents: {list(scope_dir.glob('*')) if scope_dir.exists() else 'N/A'}")


Checking path: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/SCOPE
Directory exists: True
Directory contents: [PosixPath('SCOPE/soltir_tp7.m'), PosixPath('SCOPE/bug_reports.txt'), PosixPath('SCOPE/README.md'), PosixPath('SCOPE/output'), PosixPath('SCOPE/SCOPE.m'), PosixPath('SCOPE/set_parameter_filenames.csv'), PosixPath('SCOPE/input'), PosixPath('SCOPE/run_scope_wrapper_json.m'), PosixPath('SCOPE/.readthedocs.yaml'), PosixPath('SCOPE/src'), PosixPath('SCOPE/GNU_General_Public_Licence.txt'), PosixPath('SCOPE/SCOPE.exe'), PosixPath('SCOPE/docs')]


In [ ]:
from scopeWrapper import SCOPEWrapperMultiRun

if __name__ == "__main__":
    print("\n=== Path Diagnostics ===")
    print(f"Current working directory: {Path.cwd()}")
    print(f"Checking for 'SCOPE' in: {Path.cwd()}")

    # the spectral and scalar files that will be saved in the results.json
    
    spectral_files = {
        #"Eout_spectrum": "Eout_spectrum.csv",
        #"Lo_spectrum": "Lo_spectrum.csv", 
        "fluorescence": "fluorescence.csv",
        #"sigmaF": "sigmaF.csv",
        "reflectance": "reflectance.csv",
    }
    
    scalar_files = {
        #"aPAR": "aPAR.csv",
        #"Eout": "Eout.csv",
        #"Lo": "Lo.csv",
        #"Esun": "Esun.csv",
        #"Esky": "Esky.csv",
        #"fluxes": "fluxes.csv",
        #"rad": "radiation.csv",
        #"fluorescence_scalars": "fluorescence_scalars.csv",
    }

    try:
        with SCOPEWrapperMultiRun(scalar_files=scalar_files, spectral_files=spectral_files) as scope:
            print("\n=== Running Multi-Simulation Test ===")
            # Example parameters (can be adjusted)

            # Run a simulation with multiple LAI and Cab values
            multi_params = {
                # PROSPECT leaf optical properties
                "Cab": [30.0, 40.0, 50.0],      # Chlorophyll content (μg/cm²)
                "Cca": [10.0],                  # Carotenoid content (μg/cm²)
                "Cdm": [0.012],                 # Dry matter content (g/cm²)
                "Cw": [0.009],                  # Equivalent water thickness (cm)
                "Cs": [0.0],                    # Senescent material fraction (0-1)
                "Cant": [1.0],                  # Anthocyanin content (μg/cm²)
                "N": [1.5],                     # Leaf structure parameter (-)

                # Leaf biochemical parameters
                "Vcmax25": [60.0],              # Maximum carboxylation rate at 25°C (μmol/m²/s)
                "BallBerrySlope": [8.0],        # Ball-Berry stomatal conductance slope
                "BallBerry0": [0.01],           # Ball-Berry intercept (mol/m²/s)

                # Canopy structure
                "LAI": [2.0, 3.0, 5.0],        # Leaf Area Index (m²/m²)
                "hc": [2.0],                    # Canopy height (m)
                "LIDFa": [-0.35],               # Leaf angle distribution parameter a (-)
                "LIDFb": [-0.15],               # Leaf angle distribution parameter b (-)

                # Soil parameters
                "rss": [500.0],                 # Soil respiration rate (μmol/m²/s)
                "SMC": [25.0],                  # Soil moisture content (%)
                
                # Meteorology
                "Rin": [800.0],                 # Incoming shortwave radiation (W/m²)
                "Ta": [20.0],                   # Air temperature (°C)
                "Ca": [410.0],                  # Atmospheric CO₂ concentration (ppm)

                # Fluorescence
                "fqe": [0.01, 0.03, 0.06, 0.1, 0.2], # Fluorescence quantum efficiency (-)

                # Angles
                "tts": [35.0, 0.00, 0.60],                  # Solar zenith angle (degrees)
                "tto": [0.0],                   # Observer zenith angle (degrees)
                "psi": [0.0]                    # Relative azimuth angle (degrees)
            }

            my_setoptions = {
                "simulation": 2,  # Enable Lookup-Table mode
                "calc_fluor": 1,  # Calculate fluorescence
                "soil_heat_method": 2,  # Use soil net radiation fraction
                "saveCSV": 1,  # Output CSV files
                "calc_planck": 0,
                "calc_directional": 0,
                "calc_planck": 0,
                "calc_vert_profiles": 0,
                "calc_rss_rbs": 0,
                "soil_heat_method": 2,
                "save_spectral": 1,
                "calc_xanthophyllabs": 0,
                "applTcorr": 0,
                "MoninObukhov" : 0,
                "lite": 0,
                
            }

            results_path = scope.run(multi_params, my_setoptions)

            print("Success!")
            with open(results_path, "r") as f:
                results = json.load(f)

            print("\n=== Results ===")

            print(
                f"Number of simulations in output: {results.get('num_simulations')}"
            ) 

    except Exception as e:
        print(f"Error during SCOPE execution: {e}")


=== Path Diagnostics ===
Current working directory: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project
Checking for 'SCOPE' in: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project

=== Running Multi-Simulation Test ===
=== Parameters before CSV generation ===
{
  "Cab": [
    30.0,
    40.0,
    50.0
  ],
  "Cca": [
    10.0
  ],
  "Cdm": [
    0.012
  ],
  "Cw": [
    0.009
  ],
  "Cs": [
    0.0
  ],
  "Cant": [
    1.0
  ],
  "N": [
    1.5
  ],
  "Vcmax25": [
    60.0
  ],
  "BallBerrySlope": [
    8.0
  ],
  "BallBerry0": [
    0.01
  ],
  "LAI": [
    2.0,
    3.0,
    5.0
  ],
  "hc": [
    2.0
  ],
  "LIDFa": [
    -0.35
  ],
  "LIDFb": [
    -0.15
  ],
  "rss": [
    500.0
  ],
  "SMC": [
    25.0
  ],
  "Rin": [
    800.0
  ],
  "Ta": [
    20.0
  ],
  "Ca": [
    410.0
  ],
  "fqe": [
    0.01,
    0.03,
    0.06,
    0.1,
    0.2
  ],
  "tts": [
    35.0,
    0.0,
    0.6
  ],
  "tto": [
    0.0
  ],
  "psi": [
    0.0
  ]
}

=== Generated CSV Content ===
PROS

In [5]:
print(results.keys())
print(results["spectral_outputs"].keys())
print(results["input_parameters"].keys())
# print recursive keys
def print_keys(d, prefix=""):
    for k, v in d.items():
        print(f"{prefix}{k}")
        if isinstance(v, dict):
            print_keys(v, prefix=prefix + "-")
print_keys(results)

dict_keys(['scalar_outputs', 'run_parameters', 'num_simulations', 'input_parameters', 'setoptions', 'model_parameters', 'spectral_outputs', 'wlF', 'wlS'])
dict_keys(['fluorescence', 'reflectance'])
dict_keys(['PROSPECT', 'Leaf_Biochemical', 'Leaf_Biochemical_magnani', 'Fluorescence', 'Soil', 'Canopy', 'Meteo', 'Aerodynamic', 'timeseries', 'Angles'])
scalar_outputs
run_parameters
num_simulations
input_parameters
-PROSPECT
--Cab
--Cca
--Cdm
--Cw
--Cs
--Cant
--Cp
--Cbc
--N
--rho_thermal
--tau_thermal
-Leaf_Biochemical
--Vcmax25
--BallBerrySlope
--BallBerry0
--Type
--kV
--Rdparam
--Kn0
--Knalpha
--Knbeta
-Leaf_Biochemical_magnani
--Tyear
--beta
--kNPQs
--qLs
--stressfactor
-Fluorescence
--fqe
-Soil
--spectrum
--rss
--rs_thermal
--cs
--rhos
--lambdas
--SMC
--BSMBrightness
--BSMlat
--BSMlon
-Canopy
--LAI
--hc
--LIDFa
--LIDFb
--leafwidth
--Cv
--crowndiameter
-Meteo
--z
--Rin
--Ta
--Rli
--p
--ea
--u
--Ca
--Oa
-Aerodynamic
--zo
--d
--Cd
--rb
--CR
--CD1
--Psicor
--CSSOIL
--rbs
--rwc
-timeseries


In [ ]:
import os
import logging
import shutil
import tempfile
import json
from pathlib import Path

import matlab.engine
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

# --- Configuration ---
BASE_DIR = Path("./")
MODTRAN_MATLAB_DIR = BASE_DIR / "MODTRAN5" / "matlab-modtran-5-aba70d781805"
SCOPE_OUTPUT_FILE = BASE_DIR / "synthetic_dataset" / "results.json"
OUTPUT_DIR = BASE_DIR / "output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s",
                    handlers=[logging.StreamHandler()])
logger = logging.getLogger(__name__)

# --- MODTRAN Configuration ---
MODTRAN_DIR = Path("/home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5").resolve()
logger.info(f"Looking for MODTRAN at: {MODTRAN_DIR}")
if not MODTRAN_DIR.exists():
    raise FileNotFoundError(f"MODTRAN directory not found: {MODTRAN_DIR}")

MODTRAN_EXE_DIR = MODTRAN_DIR / "bin"
if not MODTRAN_EXE_DIR.exists():
    raise FileNotFoundError(f"MODTRAN executable directory not found: {MODTRAN_EXE_DIR}")

possible_data_dirs = [MODTRAN_EXE_DIR / "DATA", MODTRAN_DIR / "DATA", MODTRAN_DIR / "data"]
MODTRAN_DATA_DIR = next((d for d in possible_data_dirs if d.exists()), None)
if MODTRAN_DATA_DIR:
    logger.info(f"Found MODTRAN data directory at: {MODTRAN_DATA_DIR}")
else:
    raise FileNotFoundError(f"MODTRAN data directory not found. Searched in: {possible_data_dirs}")

# Check for a band model file (ignored in our case)
possible_bmfiles = [MODTRAN_DATA_DIR / "p1_2013.bin", MODTRAN_DATA_DIR / "p1_2009t.bin",
                    MODTRAN_DATA_DIR / "p1_2009c.bin", MODTRAN_DATA_DIR / "p1_2009.bin"]
BMNAME_FILE = next((f for f in possible_bmfiles if f.exists()), None)
if BMNAME_FILE:
    logger.info(f"Found band model: {BMNAME_FILE.name}, but it will be ignored.")
else:
    logger.info("No band model file found. Proceeding with MODTRAN defaults.")

# --- Start MATLAB Engine ---
logger.info("Starting MATLAB Engine...")
eng = matlab.engine.start_matlab("-nojvm -nodisplay -nosplash")

# --- Configure MATLAB Paths ---
eng.addpath(str(MODTRAN_DIR.resolve()), nargout=0)
eng.addpath(eng.genpath(str(MODTRAN_MATLAB_DIR.resolve())), nargout=0)
eng.addpath(str(MODTRAN_DATA_DIR.resolve()), nargout=0)

import platform
os_type = platform.system()
if os_type == "Windows":
    modtran_exe = MODTRAN_EXE_DIR / "Mod5_win64.exe"
elif os_type == "Linux":
    modtran_exe = MODTRAN_EXE_DIR / "Mod5_linux.exe"
    if not modtran_exe.exists():
        modtran_exe = MODTRAN_EXE_DIR / "Mod5_linux.exe"

# Compute relative paths for flexibility
modtran_matlab_dir = MODTRAN_MATLAB_DIR.resolve()
modtran_exe_dir = MODTRAN_EXE_DIR.resolve()
modtran_exe_path = modtran_exe.resolve()
relative_modtran_path = os.path.relpath(str(modtran_exe_dir), str(modtran_matlab_dir))
relative_modtran_exe = os.path.relpath(str(modtran_exe_path), str(modtran_matlab_dir))

# --- Create temporary MATLAB function to override GUI dialog ---
temp_script = f"""
function [MODTRANExe, MODTRANPath] = SetMODTRANExe()
    MODTRANExe = '{modtran_exe.name}';
    MODTRANPath = '{str(MODTRAN_EXE_DIR.resolve()).replace(os.sep, '/')}/';
end
"""
temp_dir = Path(tempfile.mkdtemp())
script_path = temp_dir / "SetMODTRANExe.m"
with open(script_path, 'w') as f:
    f.write(temp_script)
eng.addpath(str(temp_dir), nargout=0)
eng.eval("Mod5.SetMODTRANExe = @SetMODTRANExe;", nargout=0)
eng.workspace["MODTRANPath"] = relative_modtran_path.replace(os.sep, '/') + "/"
eng.workspace["MODTRANExe"] = relative_modtran_exe.replace(os.sep, '/')

logger.info("Final MATLAB configuration complete.")
try:
    modtran_path = eng.workspace['MODTRANPath']
except Exception:
    modtran_path = "Not Set"
logger.info(f"MODTRANPath in MATLAB: {modtran_path}")
try:
    modtran_exe_ws = eng.workspace['MODTRANExe']
except Exception:
    modtran_exe_ws = "Not Set"
logger.info(f"MODTRANExe in MATLAB: {modtran_exe_ws}")
logger.info(f"Is Mod5.m found? {eng.exist('Mod5.m', 'file')}")
logger.info(f"Is run_MODTRAN5_AC_PAR_4RUNS.m found? {eng.exist('run_MODTRAN5_AC_PAR_4RUNS.m', 'file')}")

eng.addpath(str(MODTRAN_MATLAB_DIR.resolve()), nargout=0)
eng.addpath(eng.genpath(str((MODTRAN_MATLAB_DIR.parent / "modtran5_acd_t14").resolve())), nargout=0)
logger.info("MATLAB paths added successfully.")

# --- Helper Functions ---
def run_modtran_matlab(parms, output_dir, case_name):
    """Runs MODTRAN via MATLAB Engine with proper initialization."""
    try:
        eng.eval("warning('off', 'Mod5:setBMNAME:FileNotFound');", nargout=0)
        eng.workspace['NSTR'] = 1  # Force single-threaded execution
        eng.eval("delete(gcp('nocreate'));", nargout=0)
        matlab_parms = eng.struct()
        for key, value in parms.items():
            if key == 'ATM':
                atm_struct = eng.struct()
                for sub_key, sub_value in value.items():
                    if isinstance(sub_value, (list, np.ndarray)):
                        atm_struct[sub_key] = matlab.double([float(x) for x in sub_value])
                    elif isinstance(sub_value, (int, float)):
                        atm_struct[sub_key] = float(sub_value)
                    else:
                        atm_struct[sub_key] = sub_value
                matlab_parms[key] = atm_struct
            elif key == 'SPECTRAL':
                spectral_struct = eng.struct()
                for sub_key, sub_value in value.items():
                    if sub_key in ('V1', 'V2'):
                        spectral_struct[sub_key] = matlab.double([float(sub_value)])
                    elif isinstance(sub_value, (int, float)):
                        spectral_struct[sub_key] = float(sub_value)
                    else:
                        spectral_struct[sub_key] = sub_value
                matlab_parms[key] = spectral_struct
            else:
                if isinstance(value, (list, np.ndarray)):
                    matlab_parms[key] = matlab.double([float(x) for x in value])
                elif isinstance(value, (int, float)):
                    matlab_parms[key] = float(value)
                else:
                    matlab_parms[key] = value
        eng.cd(str(MODTRAN_MATLAB_DIR.resolve()), nargout=0)
        wvlLUT, T14, _ = eng.run_MODTRAN5_AC_PAR_4RUNS(
            str(output_dir),
            case_name,
            matlab_parms,
            'FLUO',
            'A',
            2,
            0,
            0,
            nargout=3
        )
        return np.array(wvlLUT), np.array(T14), None
    except Exception as e:
        logger.error(f"MODTRAN execution failed for {case_name}: {e}")
        return None, None, None

def interpolate_and_combine(wl_scope, data_scope, wl_modtran):
    """Interpolates SCOPE data to MODTRAN wavelengths."""
    interpolator = interp1d(wl_scope, data_scope, kind='linear', bounds_error=False, fill_value="extrapolate")
    return interpolator(wl_modtran)

def validate_modtran_params(parms):
    required_atm = ['WVL', 'WNDFNM', 'WLINC', 'MODEL', 'H1', 'SZA', 'IHAZE', 'VIS']
    for param in required_atm:
        if param not in parms['ATM']:
            raise ValueError(f"Missing required ATM parameter: {param}")
    required_top_level = ['SPECTRAL', 'GEOM']
    for param in required_top_level:
        if param not in parms:
            raise ValueError(f"Missing required top-level parameter: {param}")
    numerical_params = {
        'ATM': ['AOT', 'H1', 'SZA', 'VZA', 'GNDALT', 'WLINC','VIS'],
        'SPECTRAL': ['V1', 'V2', 'DV', 'FWHM'],
        'GEOM': ['H1ALT', 'OBSZEN'],
        'top_level': ['SFWHM', 'CO2MX', 'RAINRT', 'GNDALT','PARM1','PARM2']
    }
    for category, params in numerical_params.items():
        for param in params:
            if category == 'ATM':
                value = parms['ATM'].get(param)
            elif category == 'SPECTRAL':
                value = parms['SPECTRAL'].get(param)
            elif category == 'GEOM':
                value = parms['GEOM'].get(param)
            else:
                value = parms.get(param)
            if value is None:
                raise ValueError(f"Parameter {param} is required but has no value assigned")
            if isinstance(value, (list, np.ndarray)):
                if not all(isinstance(x, (int, float)) for x in value):
                    raise TypeError(f"Elements of parameter {param} must be numeric")
            elif not isinstance(value, (int, float)):
                raise TypeError(f"Parameter {param} must be numeric, got {type(value)}")

# --- Main Processing ---
if __name__ == "__main__":
    # Load SCOPE Results
    try:
        with open(SCOPE_OUTPUT_FILE, 'r') as f:
            scope_results = json.load(f)
    except Exception as e:
        logger.error(f"Error loading SCOPE results: {e}")
        exit(1)

    num_simulations = scope_results.get('num_simulations', 0)
    logger.info(f"Number of SCOPE simulations: {num_simulations}")

    # For each simulation, extract SCOPE conditions if they exist.
    # These SCOPE parameters (if provided) are used to override ambient conditions.
    run_params = scope_results["run_parameters"]

    modtran_results = []

    # Prepare SCOPE spectral data arrays
    wlS = np.array(scope_results['wlS'])[0]  # assume a single list
    wlF = np.array(scope_results['wlF'])[0]
    reflectance_data = np.array(scope_results['spectral_outputs']['reflectance'])
    fluorescence_data = np.array(scope_results['spectral_outputs']['fluorescence'])

    # Loop over each SCOPE simulation and each ambient condition variant.
    for i in range(num_simulations):
        # Extract SCOPE run parameters for simulation i.
        current_scope = run_params[i]
        # Override ambient conditions with SCOPE parameters if available.
        # For example, use "Canopy.hc" as ground altitude and "Angles.tts" as solar zenith angle.
        sim_gndalt = current_scope.get("Canopy.hc", 0.01)
        sim_sza = current_scope.get("Angles.tts", 30.0)
        sim_tto = current_scope.get("Angles.tto", 0.0)
        sim_psi = current_scope.get("Angles.psi", 0.0)
        # Meteo parameters such as AOT and H2O may not be present; default values are used.
        sim_aot = current_scope.get("Meteo.AOT", 0.1)
        sim_h2o = current_scope.get("Meteo.H2O", 0.5)

        # Create two ambient conditions (e.g., a base condition and one with slight modifications)
        ambient_conditions = [
            {"GNDALT": sim_gndalt, "SZA": sim_sza, "AOT": sim_aot, "H2O": sim_h2o},
            {"GNDALT": sim_gndalt, "SZA": sim_sza + 5, "AOT": sim_aot * 1.2, "H2O": sim_h2o * 1.1}
        ]
        for j, ambient in enumerate(ambient_conditions):
            case_name = f"sim_{i}_amb_{j}"
            logger.info(f"Running MODTRAN for {case_name}")
            
            # Prepare PARMS using the ambient condition and SCOPE parameters if available.
            # The PARMS dictionary contains all the input parameters for the MODTRAN simulation.
            # These parameters include top-level simulation controls, atmospheric (ATM) parameters,
            # geometric (GEOM) parameters, and spectral configuration. Many values are fixed or set to default
            # while others (e.g. GNDALT, SZA, AOT, H2O) are taken from the ambient condition.
            PARMS = {
                # Top-level simulation controls:
                'CASE': 0,                # Simulation case identifier (unused; set to 0)
                'MODTRN': 'M',            # MODTRAN mode ('M' for standard mode)
                'SPEED': 'S',             # Simulation speed option
                'LYMOLC': 0,              # Molecule-specific option (default 0)
                'MODEL': 2,               # Atmospheric model selection (e.g., 2 for midlatitude winter/summer)
                'ITYPE': 3,               # Instrument/observation type (3 corresponds to a specific sensor geometry)
                'IEMSCT': 3,              # Scattering option flag
                'IMULT': 0,               # Multiplicative flag (0 = off)
                'M1': 0,                  # Additional model parameter (set to 0)
                'M2': 0,                  # Additional model parameter (set to 0)
                'M3': 0,                  # Additional model parameter (set to 0)
                'M4': 0,                  # Additional model parameter (set to 0)
                'M5': 0,                  # Additional model parameter (set to 0)
                'M6': 0,                  # Additional model parameter (set to 0)
                'MDEF': 0,                # Model definition flag (0 = default)
                'I_RD2C': 0,              # Radiative transfer option (default 0)
                'NOPRNT': 0,              # Print output flag (0 = printing enabled)
                'TPTEMP': 20.0,           # Temperature used in Planck function (°C)
                'SURREF': 'LAMB_1',       # Surface reflection model (e.g., Lambertian reflection)
                'DIS': '1',               # Display/output option (set as string '1')
                'DISAZM': '0',            # Azimuth display flag (set to '0')
                'DISALB': ' ',            # Placeholder for albedo display option
                'NSTR': 8,                # Number of radiative transfer streams
                'SFWHM': 1.0,             # Instrument spectral response FWHM (full-width half-maximum)
                'CO2MX': 410.0,           # CO₂ mixing ratio (ppm)
                'LSUNFL': 1,              # Sun flag: include direct sun component (1 = yes)
                'LBMNAM': 0,              # Disable band model lookup (0 = disabled)
                'BMNAME': ' ',            # Band model name (empty indicates ignore)
                'IDAY': 191,              # Day of year for the simulation
                'GMTIME': 12.0,           # Greenwich Mean Time of the simulation (hours)
                'NSSALB': 0,              # Placeholder for spherical albedo option
                'IPARM': 11,              # Parameter flag that selects a particular set of parameters
                'APLUS': ' ',             # Additional parameter placeholder
                'CNOVAM': ' ',            # Additional parameter placeholder
                'ARUSS': ' ',             # Additional parameter placeholder
                'SOLCON': ' ',            # Solar constant correction flag placeholder
                'LFLTNM': ' ',            # Output file or logging filename placeholder
                'WSS': 0.0,               # Wind speed parameter (default 0)
                'WDS': 0.0,               # Wind direction parameter (default 0)
                'VIS': 23.0,              # Visibility in km (affects aerosol scattering)
                'RAINRT': 0.0,            # Rain rate (mm/hr, default 0)
                'GNDALT': ambient["GNDALT"],  # Ground altitude (km) derived from ambient condition (or SCOPE "Canopy.hc")
                'T_BEST': ' ',            # Placeholder for best-fit temperature or similar parameter

                # Atmospheric (ATM) parameters:
                'ATM': {
                    'H1': ambient["GNDALT"],       # Observer altitude (km), here set equal to ground altitude
                    'GNDALT': ambient["GNDALT"],   # Ground altitude (km)
                    'SZA': ambient["SZA"],           # Solar Zenith Angle (degrees) from ambient condition or SCOPE "Angles.tts"
                    'SAA': 180.0 - sim_psi,          # Solar Azimuth Angle (degrees); derived from SCOPE "Angles.psi"
                    'RAA': 0.0,                    # Relative Azimuth Angle (degrees; default 0)
                    'VZA': sim_tto,                # View Zenith Angle (degrees) from SCOPE "Angles.tto"
                    'MODEL': 2,                    # Atmospheric model (same as top-level MODEL)
                    'IDAY': 191,                   # Day of year for the atmospheric simulation (must match top-level IDAY)
                    'G': 0.7,                      # Asymmetry factor for scattering
                    'AOT': ambient["AOT"],         # Aerosol Optical Thickness from ambient condition or SCOPE "Meteo.AOT"
                    'CDASTM': ' ',                 # Placeholder for aerosol/storm parameters
                    'ASTMX': 0.0,                  # Maximum aerosol optical thickness (default 0)
                    'CMULT': 1.0,                  # Multiplicative factor for aerosol correction
                    'IHAZE': 1,                    # Aerosol model flag (1 = default aerosol model)
                    'H2OSTR': ambient["H2O"],       # Water vapor content from ambient condition or SCOPE "Meteo.H2O"
                    'O3STR': str(current_scope.get("Meteo.O3", 'DEFAULT')),  # Ozone content; use SCOPE value if available
                    'C_PROF': ' ',                 # Placeholder for atmospheric profile data
                    'H2OAER': ' ',                 # Placeholder for aerosol water content
                    'CDTDIR': ' ',                 # Placeholder for direct irradiance component
                    'ASTMO': ' ',                  # Placeholder for additional aerosol options
                    'AERRH': ' ',                  # Placeholder for aerosol error term
                    'IPH': 2,                      # Photon counting flag (or related parameter)
                    'ISEASN': 0,                   # Additional aerosol parameter (default 0)
                    'IVULCN': 0,                   # Additional aerosol parameter (default 0)
                    'ICSTL': 1,                    # Cloud or scattering type flag (1 = default)
                    'ICLD': 0,                     # Cloudiness flag (0 = clear skies)
                    'IVSA': 0,                     # Additional scattering parameter (default 0)
                    'WVL': [650.0, 850.0],         # Wavelength range (nm) for the simulation
                    'WNDFNM': [650.0, 850.0],      # Wavelength range for the lookup table (same as WVL)
                    'WLINC': 1.0,                  # Wavelength increment (nm); smaller values mean higher spectral resolution (more rows)
                    'VIS': 23.0,                   # Visibility (km) for atmospheric simulation (should match top-level VIS)
                },

                # Geometric (GEOM) parameters:
                'GEOM': {
                    'ITYPE': 3,                  # Geometric/observation type (3 = specific sensor configuration)
                    'H1ALT': ambient["GNDALT"],  # Sensor altitude (km) set equal to ground altitude
                    'OBSZEN': 180.0 - sim_tto,     # Observation Zenith Angle (degrees) from SCOPE "Angles.tto"
                    'TRUEAZ': sim_psi + 180.0,     # True Azimuth Angle (degrees) from SCOPE "Angles.psi"
                },

                # Additional top-level parameters:
                'PARM1': ambient["SZA"],         # Parameter 1 (typically set as the solar zenith angle)
                'PARM2': 0.0,                    # Parameter 2 (unused, set to 0)

                # Spectral configuration parameters:
                'SPECTRAL': {
                    'V1': 1e7 / 850,             # Starting wavenumber calculated from 850 nm
                    'V2': 1e7 / 650,             # Ending wavenumber calculated from 650 nm
                    'DV': 20.0,                   # Spectral resolution (cm⁻¹); lower DV means higher resolution (more rows)
                    'FWHM': 10.0,                 # Full-width half-maximum of the instrument spectral response
                    'CM_1': True                 # Flag indicating if the spectral range is in cm⁻¹ (True = yes)
                },
            }

            validate_modtran_params(PARMS)
            # Run MODTRAN simulation for the current case
            wvlLUT, T14, DV = run_modtran_matlab(PARMS, str(OUTPUT_DIR), case_name)
            if wvlLUT is None:
                logger.error(f"Skipping {case_name} due to MODTRAN error.")
                continue

            # Ensure wvlLUT is one-dimensional
            wvlLUT = np.array(wvlLUT).flatten()

            # Interpolate SCOPE outputs (reflectance and fluorescence) to MODTRAN wavelengths.
            idx_ref = i % reflectance_data.shape[0]
            R = interpolate_and_combine(wlS, reflectance_data[idx_ref, :], wvlLUT).flatten()
            F = interpolate_and_combine(wlF, fluorescence_data[idx_ref, :], wvlLUT).flatten()

            # Calculate LBOA and LWLR to obtain the apparent reflectance ρ_app per Equation 5.
            t_vals = {f't{j}': T14[:, j-1] for j in range(1, 13)}
            t1 = t_vals['t1']
            t2 = t_vals['t2']
            t3 = t_vals['t3']
            t4 = t_vals['t4'] + t_vals['t5']  # sum of t4 and t5
            t6 = t_vals['t6']
            t7 = t_vals['t7']
            t8 = t_vals['t8']
            t9 = t_vals['t9']
            t10 = t_vals['t10']
            t11 = t_vals['t11']

            LBOA = t1 * t4 * R + t1 * ((t9 * R + t10 * R + t11 * R) / (1 - R * t3)) + t1 * (t6 * F + t7 * F)
            LWLR = t1 * t4 + t1 * ((t9 * R + t10 * R + t11 * R) / (1 - R * t3))
            rho_app = LBOA / LWLR

            LTOA = t1 * t2 + (t1 * t8 * rho_app + t9 * rho_app + t10 * rho_app + t11 * rho_app + t6 * F + t7 * F) / (1 - t3 * rho_app)

            print("wvlLUT shape:", wvlLUT.shape)
            print("LTOA shape:", LTOA.shape)

            # Now both wvlLUT and LTOA should be 1D arrays with the same length.
            df = pd.DataFrame({'Wavelength': wvlLUT, 'LTOA': LTOA})
            output_file = OUTPUT_DIR / f"LTOA_{case_name}.csv"
            df.to_csv(output_file, index=False)
            logger.info(f"Saved LTOA for {case_name} to {output_file}")


            # Append simulation details to modtran_results for updating results.json later.
            modtran_info = {
                "case_name": case_name,
                "PARMS": PARMS,
                "wavelength": wvlLUT.flatten().tolist(),
                "T14": T14.tolist(),
                "DV": DV,
                "rho_app": rho_app.flatten().tolist(),
                "LTOA": LTOA.flatten().tolist()
            }
            modtran_results.append(modtran_info)

    # Quit MATLAB engine and clean up temporary directory
    eng.quit()
    shutil.rmtree(temp_dir)
    logger.info("MATLAB Engine quit and temporary files removed.")

    # Update SCOPE results JSON with MODTRAN simulation details.
    scope_results.setdefault("modtran_simulations", [])
    scope_results["modtran_simulations"].extend(modtran_results)
    with open(SCOPE_OUTPUT_FILE, "w") as f:
        json.dump(scope_results, f, indent=2)
    logger.info(f"Updated results saved to {SCOPE_OUTPUT_FILE}")

    logger.info("Processing complete.")


2025-03-18 19:21:17,406 [INFO] Looking for MODTRAN at: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5
2025-03-18 19:21:17,407 [INFO] Found MODTRAN data directory at: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/DATA
2025-03-18 19:21:17,408 [INFO] Found band model: p1_2009t.bin, but it will be ignored.
2025-03-18 19:21:17,408 [INFO] Starting MATLAB Engine...
2025-03-18 19:21:20,126 [INFO] Final MATLAB configuration complete.
2025-03-18 19:21:20,130 [INFO] MODTRANPath in MATLAB: ../bin/
2025-03-18 19:21:20,134 [INFO] MODTRANExe in MATLAB: ../bin/Mod5_linux.exe
2025-03-18 19:21:20,141 [INFO] Is Mod5.m found? 2.0
2025-03-18 19:21:20,147 [INFO] Is run_MODTRAN5_AC_PAR_4RUNS.m found? 2.0
2025-03-18 19:21:20,186 [INFO] MATLAB paths added successfully.
2025-03-18 19:21:20,220 [INFO] Number of SCOPE simulations: 135
2025-03-18 19:21:20,228 [INFO] Running MODTRAN for sim_0_amb_0


--- Starting run_MODTRAN5_AC_PAR_4RUNS at 18-Mar-2025 19:21:20 ---
MODTRAN5 - MIT 4 RUNS method (parallel version)
Initializing simulation parameters...
Cleared previous MODTRAN classes.
Atmospheric and geometric parameters set.
Loading tp5 base file for SENSOR: FLUO
TP5 base file loaded.
Base file parameters configured.
**************************************
MODTRAN5 is running in 5cm-1 configuration
**************************************
Output directory created at output/T14-Modtran
Initialized MODTRAN spectral output array.
Writing TP5 base file to: output/T14-Modtran/sim_0_amb_0.tp5
TP5 base file written.
Initializing MODTRAN5 cases...
MODTRAN5 cases initialized.
Loading MODTRANExe settings...
Using MODTRANPath: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/
Using MODTRANExe: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_linux.exe
Starting parallel pool (parpool) using the 'local' profile ...
Connected to parallel pool with 4 

2025-03-18 19:21:30,753 [INFO] Saved LTOA for sim_0_amb_0 to output/LTOA_sim_0_amb_0.csv
2025-03-18 19:21:30,753 [INFO] Running MODTRAN for sim_0_amb_1


Worker 1: Removed temporary directory sim_0_amb_0_BOA05.
Worker 4: Removed temporary directory sim_0_amb_0_TOA10.
Worker 2: Removed temporary directory sim_0_amb_0_BOA10.
Worker 3: Removed temporary directory sim_0_amb_0_TOA00.
Calculating atmospheric functions from simulation output...
Final output saved to output/T14fnct_sim_0_amb_0.mat
--- run_MODTRAN5_AC_PAR_4RUNS finished at 18-Mar-2025 19:21:30 ---
wvlLUT shape: (241,)
LTOA shape: (241,)
Parallel pool using the 'Processes' profile is shutting down.
--- Starting run_MODTRAN5_AC_PAR_4RUNS at 18-Mar-2025 19:21:32 ---
MODTRAN5 - MIT 4 RUNS method (parallel version)
Initializing simulation parameters...
Cleared previous MODTRAN classes.
Atmospheric and geometric parameters set.
Loading tp5 base file for SENSOR: FLUO
TP5 base file loaded.
Base file parameters configured.
**************************************
MODTRAN5 is running in 5cm-1 configuration
**************************************
Output directory created at output/T14-Modtran

2025-03-18 19:21:40,161 [INFO] Saved LTOA for sim_0_amb_1 to output/LTOA_sim_0_amb_1.csv
2025-03-18 19:21:40,162 [INFO] Running MODTRAN for sim_1_amb_0


...

Worker 2: Removed temporary directory sim_3_amb_1_BOA10.
Worker 1: Removed temporary directory sim_3_amb_1_BOA05.
Worker 4: Removed temporary directory sim_3_amb_1_TOA10.
Worker 3: Removed temporary directory sim_3_amb_1_TOA00.
Calculating atmospheric functions from simulation output...
Final output saved to output/T14fnct_sim_3_amb_1.mat
--- run_MODTRAN5_AC_PAR_4RUNS finished at 18-Mar-2025 19:22:35 ---
wvlLUT shape: (241,)
LTOA shape: (241,)
Parallel pool using the 'Processes' profile is shutting down.
--- Starting run_MODTRAN5_AC_PAR_4RUNS at 18-Mar-2025 19:22:36 ---
MODTRAN5 - MIT 4 RUNS method (parallel version)
Initializing simulation parameters...
Cleared previous MODTRAN classes.
Atmospheric and geometric parameters set.
Loading tp5 base file for SENSOR: FLUO
TP5 base file loaded.
Base file parameters configured.
**************************************
MODTRAN5 is running in 5cm-1 configuration
**************************************
Output directory created at output/T14-Modtran

2025-03-18 19:22:44,483 [INFO] Saved LTOA for sim_4_amb_0 to output/LTOA_sim_4_amb_0.csv
2025-03-18 19:22:44,484 [INFO] Running MODTRAN for sim_4_amb_1


Worker 2: Removed temporary directory sim_4_amb_0_BOA10.
Worker 1: Removed temporary directory sim_4_amb_0_BOA05.
Worker 3: Removed temporary directory sim_4_amb_0_TOA00.
Worker 4: Removed temporary directory sim_4_amb_0_TOA10.
Calculating atmospheric functions from simulation output...
Final output saved to output/T14fnct_sim_4_amb_0.mat
--- run_MODTRAN5_AC_PAR_4RUNS finished at 18-Mar-2025 19:22:44 ---
wvlLUT shape: (241,)
LTOA shape: (241,)
Parallel pool using the 'Processes' profile is shutting down.
--- Starting run_MODTRAN5_AC_PAR_4RUNS at 18-Mar-2025 19:22:45 ---
MODTRAN5 - MIT 4 RUNS method (parallel version)
Initializing simulation parameters...
Cleared previous MODTRAN classes.
Atmospheric and geometric parameters set.
Loading tp5 base file for SENSOR: FLUO
TP5 base file loaded.
Base file parameters configured.
**************************************
MODTRAN5 is running in 5cm-1 configuration
**************************************
Output directory created at output/T14-Modtran

2025-03-18 19:22:53,488 [INFO] Saved LTOA for sim_4_amb_1 to output/LTOA_sim_4_amb_1.csv
2025-03-18 19:22:53,489 [INFO] Running MODTRAN for sim_5_amb_0


Worker 3: Removed temporary directory sim_4_amb_1_TOA00.
Worker 4: Removed temporary directory sim_4_amb_1_TOA10.
Worker 2: Removed temporary directory sim_4_amb_1_BOA10.
Worker 1: Removed temporary directory sim_4_amb_1_BOA05.
Calculating atmospheric functions from simulation output...
Final output saved to output/T14fnct_sim_4_amb_1.mat
--- run_MODTRAN5_AC_PAR_4RUNS finished at 18-Mar-2025 19:22:53 ---
wvlLUT shape: (241,)
LTOA shape: (241,)
Parallel pool using the 'Processes' profile is shutting down.
--- Starting run_MODTRAN5_AC_PAR_4RUNS at 18-Mar-2025 19:22:54 ---
MODTRAN5 - MIT 4 RUNS method (parallel version)
Initializing simulation parameters...
Cleared previous MODTRAN classes.
Atmospheric and geometric parameters set.
Loading tp5 base file for SENSOR: FLUO
TP5 base file loaded.
Base file parameters configured.
**************************************
MODTRAN5 is running in 5cm-1 configuration
**************************************
Output directory created at output/T14-Modtran